# Practical example using an Exhaust gas temperature simulator

## Necessary setup for Google Colab & imports

In [ ]:
! git clone https://github.com/ml-kiwi-com/ml-prague.git
! pip install -r ml-prague/requirements.txt

In [ ]:
%cd ml-prague

In [ ]:
import src.numerical_simulator as simulator
from sklearn import preprocessing
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import  ConstantKernel, RBF
from src import plots
from src import utils
from sklearn.model_selection import LeaveOneOut
import pandas as pd
from scipy.stats import norm


## The Simulator

Simulated variable: **Maximal exhaust gas temperature during take off of an airplane. [K]**

Input variables:

| name | description | unit |
| --- | --- | --- |
| *oat* | Outside air temperature at take off | [K] |
| *max_speed* | Maximal flight speed at take off | [km/h] |
| *altitude* | Airport altitude at take off | [m] |
| *humidity* | Air humidity at take off | [%] |
| *air_density* | Air density at take off | [kg/m3] |

### Simulator settings

In [ ]:
num_sim = 200
egt_threshold = 1250

### First, let's generate a dataframe of input values

In [ ]:
X = simulator.generate_input_data(num_sim)
X.sample(5).round(2)

### Now we can use our complex and heavy simulator to generate the true values of the target variable
Note the potential simulation time.

In [ ]:
y = simulator.simulate_max_egt(X)
print(f"Output range is [{round(y.min(),2)} ; {round(y.max(), 2)}]")
y.sample(5).rename('egt').to_frame().round(2)


### Let's identify dangerous observations where temperature is too high

In [ ]:
nbr_risk_obs = (y > egt_threshold).sum()
print(f"Number of obs that are potentially dangerous: {nbr_risk_obs} out of {num_sim}\n")
rng_max = y.argmax()
x_max = X.loc[rng_max,]
print("The worst observation of all:")
x_max.round(2).to_frame()

In [ ]:
X_risk = X[y > egt_threshold]
print("Only oat and air_density are influential on the risk area.")
X_risk.describe()

## The Gaussian Process Regression
First we need to scale the input data before applying the Kriging algorithm.

In [ ]:
scaler = preprocessing.StandardScaler().fit(X)
X_scaled = scaler.transform(X)

Test data.

In [ ]:
num_sim_test = 10000
X_test = utils.generate_input_data(num_sim_test)
y_test = utils.simulate_max_egt(X_test)
X_test_scaled = scaler.transform(X_test)

Then we create a grid on which we will be building the predictor.

In [ ]:
grid_1D = np.linspace(0, 1, 11)
X_grid_scaled = np.array(np.meshgrid(*([grid_1D]*5))).T.reshape(-1, 5)

Now we can build and fit the Kriging model.

In [ ]:
kernel = ConstantKernel(1.0, constant_value_bounds=(1, 1e6)) * RBF(
    np.repeat(1.0, X.shape[1]), length_scale_bounds=(0.1, 1e4)
)
gaussian_process = GaussianProcessRegressor(
    kernel=kernel, random_state=0, n_restarts_optimizer=5, normalize_y=True
)
gaussian_process.fit(X_scaled, y)
gaussian_process.kernel_.get_params()


With the trained GPR, we can now predict.

In [ ]:
y_hat, y_std = gaussian_process.predict(X_scaled, return_std=True)

Now let's visualize the Kriging estimators.

We create 1D slices of the 5D space. For each slice, we select one observation at which we keep 4 dimensions constant.
We then inspect what will happen to our target's prediction if we change the values of the remaining dimension.

In [ ]:
rng = np.random.RandomState(1)
vis_index = rng.choice(X_scaled.shape[0], size=1, replace=False)
x_predict = X_scaled[vis_index, :]
y_predict = y[vis_index]
x_axis = np.linspace(-2, 2, 101)
plots.show_multiD_slices_plots(X, x_predict, x_axis, gaussian_process, y_predict)

Model evaluation.

In [ ]:
y_range = 100
y_test_pred = gaussian_process.predict(X_test_scaled)
mse_test = (((y_test - y_test_pred)/y_range)**2)
mae_test = abs(y_test - y_test_pred)/y_range

rmse_rounded = round(np.mean(mse_test) ** 0.5 * 100, 2)
mae_rounded = round(np.mean(mae_test) * 100, 2)
print(f"Leave-one-out error metrics: \n"
      f"    RMSE: {rmse_rounded}% \n"
      f"    MAE:  {mae_rounded}%.")

## Leave-One-Out Approach

As every observation is precious, we do not want to throw away any, which means that we cannot do a standard train-test set approach.

First let's generate some simulations, scale them, and build the grid.

In [ ]:
num_sim = 50
X = utils.generate_input_data(num_sim)
y = utils.simulate_max_egt(X)
print(f"Design space dimensions: {X.shape}")

scaler = preprocessing.StandardScaler().fit(X)
X_scaled = scaler.transform(X)

grid_1D = np.linspace(0, 1, 11)
X_grid_scaled = np.array(np.meshgrid(*([grid_1D] * 5))).T.reshape(-1, 5)


Build Kriging model, train it, and store hyperparameters for later use.

In [ ]:
kernel = ConstantKernel(1.0, constant_value_bounds=(1, 1e6)) * RBF(
    np.repeat(1.0, X.shape[1]), length_scale_bounds=(1, 1e4)
)
gaussian_process = GaussianProcessRegressor(
    kernel=kernel, random_state=0, n_restarts_optimizer=5, normalize_y=True
)
gaussian_process.fit(X_scaled, y)

k1_LOO, k2_LOO = (
    gaussian_process.kernel_.k1.constant_value,
    gaussian_process.kernel_.k2.length_scale,
)
kernel_LOO = ConstantKernel(k1_LOO, constant_value_bounds="fixed") * RBF(
    length_scale=k2_LOO, length_scale_bounds="fixed"
)
gaussian_process_LOO = GaussianProcessRegressor(kernel=kernel_LOO, optimizer=None)


Create the collection of LOO train/test and build Kriging model in each case.

In [ ]:
loo = LeaveOneOut()
loo.split(X_scaled, y)
y_pred_list = []
mse_list = []
mae_list = []
for i, (train_index, test_index) in enumerate(loo.split(X_scaled)):
     X_train = X_scaled[train_index]
     X_test = X_scaled[test_index]
     y_train = y[train_index]
     y_test = y[test_index]
     gaussian_process_LOO.fit(X_train, y_train)
     y_test_pred = gaussian_process_LOO.predict(X_test, return_std=False)
     y_pred_list.append(y_test_pred[0])
     mse_list.append(((y_test - y_test_pred) / y_test)  ** 2)
     mae_list.append(abs((y_test - y_test_pred) / y_test))

rmse_rounded = round(np.mean(mse_list) ** 0.5 * 100, 2)
mae_rounded = round(np.mean(mae_list) * 100, 2)
print(f"Leave-one-out error metrics: \n"
      f"    relative RMSE: {rmse_rounded}% \n"
      f"    relative MAE:  {mae_rounded}%")

## Sequential Design - Local Improvement

Generate inputs, scale data, make their copy for enrichment, generate grid.

Generate test data (larger set).

In [ ]:
y_range = 100
num_sim = 30
X = utils.generate_input_data(num_sim)
y = utils.simulate_max_egt(X)

scaler = preprocessing.StandardScaler().fit(X)
X_scaled = scaler.transform(X)

X_rich = X_scaled.copy()
y_rich = y.copy()

num_sim_test = 10000
X_test = utils.generate_input_data(num_sim_test)
y_test = utils.simulate_max_egt(X_test)
X_test_scaled = scaler.transform(X_test)

grid_1D = np.linspace(-2, 2, 11)
X_grid_scaled = np.array(np.meshgrid(*([grid_1D] * 5))).T.reshape(-1, X.shape[1])


Initiate loop to add points to the design sequentially
IMPORTANT: do not recalculate the hyperparameters after each added point but only after each added batch

In [ ]:
batch_nbr = 6
batch_size = 5
for iter_batch in range(batch_nbr):
    # Build Kriging model - estimate hyperparameters for the whole batch
    if iter_batch == 0:
        kernel = ConstantKernel(1.0, constant_value_bounds=(1, 1e6)) * RBF(np.repeat(1.0, X.shape[1]),
                                                                           length_scale_bounds=(1, 1e6))
    # In case a gpr has already be fitted, re-use last hyperparameters as initial steps for upcoming fitting
    else:
        kernel = ConstantKernel(k1_batch, constant_value_bounds=(1, 1e6)) * RBF(k2_batch,
                                                                           length_scale_bounds=(1, 1e6))

    gaussian_process = GaussianProcessRegressor(kernel=kernel,
                                                random_state=0, n_restarts_optimizer=5, normalize_y=True)
    gaussian_process.fit(X_rich, y_rich)

    # Save resulting hyperparameters to fasten the model build in upcoming batch of points
    k1_batch, k2_batch = gaussian_process.kernel_.k1.constant_value, gaussian_process.kernel_.k2.length_scale
    kernel_batch = ConstantKernel(k1_batch, constant_value_bounds="fixed") * RBF(length_scale=k2_batch,
                                                                             length_scale_bounds="fixed")
    gaussian_process_batch = GaussianProcessRegressor(kernel=kernel_batch, optimizer=None)
    gaussian_process_batch.fit(X_rich, y_rich)
    kriging_mean, kriging_std = gaussian_process_batch.predict(X_grid_scaled, return_std=True)

    # add "batch_size" points according to prediction with the same Kriging hyperparameters
    for iter in range(batch_size):
        # Pick point with highest Kriging error and add it to the design
        rng_max = kriging_std.argmax()
        new_x = X_grid_scaled[rng_max]
        new_x_unscaled = scaler.inverse_transform(new_x.reshape(1, 5))
        new_y = utils.simulate_max_egt(pd.DataFrame(new_x_unscaled, columns=X.columns))
        X_rich = np.append(X_rich, [new_x], axis=0)
        y_rich = np.append(y_rich, new_y)

        # Re-run Kriging model w/ enriched design
        gaussian_process_batch.fit(X_rich, y_rich)
        kriging_mean, kriging_std = gaussian_process_batch.predict(X_grid_scaled, return_std=True)


## Sequential Design - EGO

First, let's setup the data, scale the data and make copy for enrichment. Build grid.

In [ ]:
EGT_threshold = 1250

num_sim = 30
X = utils.generate_input_data(num_sim)
y = utils.simulate_max_egt(X)
print(f"Design space dimensions: {X.shape}")

scaler = preprocessing.StandardScaler().fit(X)
X_scaled = scaler.transform(X)

X_rich = X_scaled.copy()
y_rich = y.copy()

grid_1D = np.linspace(-2, 2, 11)
X_grid_scaled = np.array(np.meshgrid(*([grid_1D] * 5))).T.reshape(-1, X.shape[1])


Set te current maximum.

In [ ]:
current_max_x = X_rich[y_rich.argmax()]
current_max = y_rich.max()

print(f"Current max is y={current_max} for X:")
current_max_x

Build Kriging model, estimate hyperparameters, improve maximum sequentially.

In [ ]:
n_iter = 20

for iter in range(n_iter):
    if iter == 0:
        kernel = ConstantKernel(1.0, constant_value_bounds=(1, 1e6)) * RBF(
            np.repeat(1.0, X.shape[1]), length_scale_bounds=(1, 1e6)
        )
    else:
        kernel = ConstantKernel(k1, constant_value_bounds=(1, 1e6)) * RBF(
            k2, length_scale_bounds=(1, 1e6)
        )
    gaussian_process = GaussianProcessRegressor(
        kernel=kernel, random_state=0, n_restarts_optimizer=20, normalize_y=True
    )
    gaussian_process.fit(X_rich, y_rich)
    mean_prediction, std_prediction = gaussian_process.predict(X_grid_scaled, return_std=True)

    k1, k2 = (
        gaussian_process.kernel_.k1.constant_value,
        gaussian_process.kernel_.k2.length_scale,
    )

    ei_x = utils.expected_improvement(mean_prediction, std_prediction, current_max)
    rng_max_ei = ei_x.argmax()
    new_x = X_grid_scaled[rng_max_ei]
    new_x_unscaled = scaler.inverse_transform(new_x.reshape(1, 5))
    new_y = utils.simulate_max_egt(pd.DataFrame(new_x_unscaled, columns=X.columns))
    X_rich = np.append(X_rich, [new_x], axis=0)
    y_rich = np.append(y_rich, new_y)

    current_max_x = X_rich[y_rich.argmax()]
    current_max = y_rich.max()

print(f"Final maximum: \n" f"    X: {current_max_x}% \n" f"    y:  {current_max}%.")


Buld Kriging model for the whole batch and check all observations that are above the threshold.

In [ ]:
kernel = ConstantKernel(1.0, constant_value_bounds=(1, 1e6)) * RBF(
    np.repeat(1.0, X.shape[1]), length_scale_bounds=(1, 1e6)
)
gaussian_process = GaussianProcessRegressor(
    kernel=kernel, random_state=0, n_restarts_optimizer=5, normalize_y=True
)
gaussian_process.fit(X_rich, y_rich)
mean_prediction, std_prediction = gaussian_process.predict(X_grid_scaled, return_std=True)

bad_X_pred = pd.DataFrame(X_grid_scaled[mean_prediction > EGT_threshold], columns=X.columns)

indices_bad_prob = (1 - norm.cdf((EGT_threshold - mean_prediction) / std_prediction)) > 0.2
bad_X_prob = pd.DataFrame(X_grid_scaled[indices_bad_prob], columns=X.columns)

display(bad_X_pred.describe())
display(bad_X_prob.describe())


## Link to the theoretical part

https://colab.research.google.com/github/ml-kiwi-com/ml-prague/blob/main/01_theoretical_part.ipynb